In [1]:
#Libraries
import os
import pydicom
import matplotlib.pyplot as plt
from lungmask import mask
import SimpleITK as sitk
import numpy as np
from scipy.ndimage.filters import median_filter
from scipy.ndimage import affine_transform

C:\Users\Nimesha\AppData\Local\Temp\ipykernel_20280\3183033577.py:8: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import median_filter


In [2]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import cv2

In [3]:
def transform_to_hu(medical_image, image):
    intercept = medical_image.RescaleIntercept
    slope = medical_image.RescaleSlope
    hu_image = image * slope + intercept

    return hu_image

def tilt_correction(medical_image,image):
    # Calculate the tilt angle using the metadata
    tilt_angle = medical_image.SliceLocation - medical_image.ImagePositionPatient[2]
  
    # Define the transformation matrix
    theta = tilt_angle * np.pi / 180
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    M = np.array([[cos_theta, -sin_theta, 0],
                  [sin_theta, cos_theta, 0],
                  [0, 0, 1]])

    # Perform the affine transformation
    transformed_data = affine_transform(image , M)

    return transformed_data

def crop_image(image, display=False):

    # Define the crop boundaries
    top = 50
    bottom = image.shape[0] - 50
    left = 50
    right = image.shape[1] - 50

    # Crop the pixel data
    cropped_data = image[top:bottom, left:right]

    return cropped_data
   

In [4]:
def get_mask(filename, plot_mask=False, return_val=False):

    input_image = sitk.ReadImage(filename)
    mask_out = mask.apply(input_image)[0]  #default model is U-net(R231)
    # if plot_mask: 
    #     fig = plt.figure(figsize=(4, 4))
    #     # plt.imshow(mask_out)
        
    #     # plt.imshow(mask_out)
    if return_val:
        return mask_out

In [5]:
def preprocess_images(img,dicom_image):

    hu_image = transform_to_hu(dicom_image, img)

    # medianl filter for noise reduction 
    # Apply the median filter with a kernel size of 3x3
    filtered_image = median_filter(hu_image, size=(3, 3))

    return filtered_image


def change_dimensions(img):


    #crop unnecessary parts from image
    croppedImage = crop_image(img)

    # Resize the image to size (28, 28)
    resized_image = np.resize(croppedImage, (28, 28))

    # Convert the numpy array to a PyTorch tensor
    tensor = torch.from_numpy(resized_image)

    # Reshape the tensor to have dimensions (1, 28, 28) to represent one color channel
    tensor = tensor.reshape((1, 28, 28))
    print(tensor.shape)

    # Normalize the tensor to have values between 0 and 1
    tensor = tensor.float() / 255.0

    # Add two more dimensions to represent the other two color channels (RGB)
    tensor = tensor.repeat(3, 1, 1)

    # Transpose the tensor to have dimensions (3, 28, 28)
    tensor = torch.transpose(tensor, 0, 2)
    tensor = torch.transpose(tensor, 1, 2)

    print(type(tensor))
    

In [6]:
from keras.applications.vgg16 import VGG16
import tensorflow as tf


# Load the pre-trained VGG16 model with ImageNet weights
VGG16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the pre-trained model so that they are not retrained
for layer in VGG16_model.layers:
    layer.trainable = False


In [7]:

def read_images(folder_path):

    # Initialize dictionaries to store support features and prototypes
    features = {}

    # Initialize a figure with a 4x5 grid of subplots
    # fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(16, 12))

    for i, sub_folder in enumerate(os.listdir(folder_path)):
            
            sub_folder_path = os.path.join(folder_path+'/'+ sub_folder)
        
            label =sub_folder[-1]
            print(label)

            # Initialize list to store support features for current class
            features[label] = []
                
            for j, dcm_file in enumerate(os.listdir(sub_folder_path)):
                
                file_path = os.path.join(sub_folder_path+'/'+ dcm_file)
                
                dicom_image = pydicom.dcmread(file_path)
                img = dicom_image.pixel_array

                cleaned_image = preprocess_images(img,dicom_image)
                masked_img=get_mask(file_path,plot_mask=True,return_val=True)
                mask_on_orginal = cleaned_image*masked_img

                resized_img = cv2.resize(mask_on_orginal, (224, 224))

                normalized_img = resized_img.astype('float32') / 255

                if len(normalized_img.shape) == 2:
                    normalized_img = np.stack([normalized_img] * 3, axis=-1)

                input_img = np.expand_dims(normalized_img, axis=0)   


                #apply vgg16 model for  extract features 
                feature_vec = VGG16_model.predict(input_img)
               
                # Append features to support_features
                features[label].append(feature_vec[0]) 

 
    return features



# Set the path to the folder containing the sub folders 
support_folder_path = 'C:/Users/Nimesha/Documents/MSC_RESEARCH/IMAGES/Support_Set'
query_folder_path   =  'C:/Users/Nimesha/Documents/MSC_RESEARCH/IMAGES/Query_Set'


support_features=read_images(support_folder_path)
query_features=read_images(query_folder_path)

print(type(support_features))
print(type(query_features))



# Initialize empty dictionary to store prototypes for each class
prototypes = {}

# Calculate the prototype for each class
for label, features in support_features.items():
    
    # Convert list of feature vectors to numpy array
    features_array = np.array(features)
    
    # Calculate mean of feature vectors
    prototype = np.mean(features_array, axis=0)
    # Store prototype for current class in dictionary
    prototypes[label] = prototype

print(type(prototypes))



import numpy as np

# Iterate over each query image and calculate the distance to each prototype
for query_label, query_features in query_features.items():

    print(query_label)
    # Convert list of feature vectors to numpy array
    query_features_array = np.array(query_features)
    
    # Initialize an empty list to store the distances between the query image and each prototype
    distances = []
    
    for prototype_label, prototype in prototypes.items():
        # Calculate the Euclidean distance between the query image features and the prototype features
        distance = np.linalg.norm(query_features_array - prototype, axis=1)
        
        # Append the distance to the list of distances
        distances.append(distance)
    
    # Convert the list of distances to a 2D numpy array
    distances_array = np.array(distances)
    
    # The classification score is -1 * the distance, so invert the distances
    scores = -1 * distances_array
    
    # The predicted class is the one with the highest score
    predicted_label = np.argmax(scores)
    
    # Print the predicted label for the query image
    print(f"Query image with label {query_label} is predicted to belong to class {predicted_label}")


A
INFO:root:No GPU support available, will use CPU. Note, that this is significantly slower!


100%|██████████| 2/2 [00:00<?, ?it/s]


1/1 [==============================] - 0s 188ms/step
INFO:root:No GPU support available, will use CPU. Note, that this is significantly slower!


100%|██████████| 2/2 [00:00<?, ?it/s]


1/1 [==============================] - 0s 94ms/step
INFO:root:No GPU support available, will use CPU. Note, that this is significantly slower!


...

1/1 [==============================] - 0s 94ms/step
E
INFO:root:No GPU support available, will use CPU. Note, that this is significantly slower!


100%|██████████| 2/2 [00:00<00:00, 2000.14it/s]


1/1 [==============================] - 0s 93ms/step
G
INFO:root:No GPU support available, will use CPU. Note, that this is significantly slower!


100%|██████████| 2/2 [00:00<?, ?it/s]


1/1 [==============================] - 0s 98ms/step
<class 'dict'>
<class 'dict'>


<class 'dict'>


Query image with label A is predicted to belong to class 1
Query image with label B is predicted to belong to class 1
Query image with label E is predicted to belong to class 1
Query image with label G is predicted to belong to class 1
